In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import LinearSegmentedColormap
from scipy import stats
plt.style.use('ggplot')
%matplotlib inline

In [2]:
nor_counts=pd.read_table('normalized_counts.txt')
col_new=['geneID', 'geneLength', 'geneProduct','185-1','WT-1','185-2','185-3','WT-2','WT-3']
nor_counts.columns=col_new
nor_counts=nor_counts[['geneID','geneLength','geneProduct','WT-1','WT-2','WT-3','185-1','185-2','185-3']]
nor_counts['WT_mean']=np.apply_along_axis(np.mean,1,nor_counts[['WT-1','WT-2','WT-3']])
nor_counts['185_mean']=np.apply_along_axis(np.mean,1,nor_counts[['185-1','185-2','185-3']])

In [4]:
gene_id_1=pd.read_table('Gene_id_1_1000.txt')
gene_id_2=pd.read_table('Gene_id_1001_2000.txt')
gene_id_3=pd.read_table('Gene_id_2001_3000.txt')
gene_id_4=pd.read_table('Gene_id_3001_3031.txt')
gene_id_main=[gene_id_1,gene_id_2,gene_id_3,gene_id_4]
gene_id_main=pd.concat(gene_id_main)
gene_id_main=gene_id_main.drop(['Batch1','Genome Name','Unnamed: 7','Genome ID'],axis=1)

In [5]:
merged=nor_counts.merge(gene_id_main,how='left',left_on='geneID',right_on='Gene ID')
df=merged.drop(['Gene Product Name','Gene ID','geneLength'],axis=1)
col_order=['geneID','Locus Tag','Gene Symbol','geneProduct','WT-1', 'WT-2', 'WT-3', '185-1', '185-2',
       '185-3', 'WT_mean', '185_mean' ]
df=df[col_order]

In [6]:
df.head(5)

,geneID,Locus Tag,Gene Symbol,geneProduct,WT-1,WT-2,WT-3,185-1,185-2,185-3,WT_mean,185_mean
0,650468844,Clo1313_0001,dnaA,chromosomal replication initiator protein DnaA,95.68,68.16,33.52,12.22,13.38,14.04,65.786667,13.213333
1,650468845,Clo1313_0002,NaN,"DNA polymerase III, beta subunit (EC 2.7.7.7)",91.89,65.97,34.76,7.39,13.24,7.93,64.206667,9.520000
2,650468846,Clo1313_0003,NaN,S4 domain protein YaaA,31.74,23.39,12.33,6.55,7.83,12.05,22.486667,8.810000
3,650468847,Clo1313_0004,recF,DNA replication and repair protein RecF,21.31,13.09,9.20,1.22,0.00,1.12,14.533333,0.780000
4,650468848,Clo1313_0005,NaN,hypothetical protein,221.06,242.11,267.31,103.35,82.30,130.65,243.493333,105.433333


### Add fold change columns to the data frame

In [7]:
df['log2(WT)']=np.log2(df['WT_mean'])
df['log2(185)']=np.log2(df['185_mean'])
df['log2(FC)']=df['log2(185)']-df['log2(WT)']

C:\Users\miluo\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log2
  """Entry point for launching an IPython kernel.
C:\Users\miluo\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log2
  


In [8]:
# remove genes with detectable transcription, which leads to numeric instability in log2 fold change computation.
# this filtered data frame should be used in picking out the top 10 gene list
# df_filtered=df[np.logical_and(df['WT_mean']>2,df['185_mean']>2)]
# df_filtered=df_filtered[['geneID','Locus Tag','Gene Symbol','geneProduct','WT_mean','185_mean','log2(FC)']]

In [9]:
# remove rows with nan or inf output in log2(FC)
row_nan_inf=np.logical_or(np.isnan(df['log2(FC)']),np.isinf(df['log2(FC)']))
df=df[~row_nan_inf]

In [10]:
df.head(5)

,geneID,Locus Tag,Gene Symbol,geneProduct,WT-1,WT-2,WT-3,185-1,185-2,185-3,WT_mean,185_mean,log2(WT),log2(185),log2(FC)
0,650468844,Clo1313_0001,dnaA,chromosomal replication initiator protein DnaA,95.68,68.16,33.52,12.22,13.38,14.04,65.786667,13.213333,6.039723,3.723923,-2.315801
1,650468845,Clo1313_0002,NaN,"DNA polymerase III, beta subunit (EC 2.7.7.7)",91.89,65.97,34.76,7.39,13.24,7.93,64.206667,9.520000,6.004651,3.250962,-2.753690
2,650468846,Clo1313_0003,NaN,S4 domain protein YaaA,31.74,23.39,12.33,6.55,7.83,12.05,22.486667,8.810000,4.490998,3.139142,-1.351856
3,650468847,Clo1313_0004,recF,DNA replication and repair protein RecF,21.31,13.09,9.20,1.22,0.00,1.12,14.533333,0.780000,3.861294,-0.358454,-4.219748
4,650468848,Clo1313_0005,NaN,hypothetical protein,221.06,242.11,267.31,103.35,82.30,130.65,243.493333,105.433333,7.927738,6.720187,-1.207551


### Compute adjusted p-value (Welch-t-test, Benjamini-Hochberg correction)
Using unpooled error to calculate the unadjusted p-value
There are several ways to adjust p-values, Bonferroni correction will simply take the unadjusted p-value and times the number of hypothesis. Alternative, FDR-adjusted, aka Benjimini & Hochberg correction, can also be used. For multiple comparison with large m, Bonferroni tend to be too severe. Benjimini & Hochberg method is used herein.

In [11]:
def calculate_mean_and_variance(df):
    """
    Imput df as a pandas dataframe (m x n).
    Each row of df is a different gene of interest, each column of df is a repeat measurement.
    Output mean (m x 1) and variance (m x 1).
    """
    df_np=np.array(df)
    m=df.shape[0]
    n=df.shape[1]
    df_mean=np.apply_along_axis(np.mean,1,df_np).reshape((m,1))
    df_var=np.sum((df_np-df_mean)**2,axis=1)/n
    return df_mean,df_var

In [12]:
def calculate_t_stat(interest,control):
    """
    Take in two dataframes of same shape (m x n)
    Each row of df is a different gene of interest, each column of df is a repeat measurement.
    Calculate the t-stastic (m x 1) and degree of freedom (m x 1), assuming equal sample size, unequal variancies (Welch t-test)
    Output in the form of numpy array.
    """
    assert interest.shape==control.shape
    m=interest.shape[0]
    n=interest.shape[1]
    int_mean,int_var=calculate_mean_and_variance(interest)
    ctl_mean,ctl_var=calculate_mean_and_variance(control)
    variance=np.sqrt((int_var/n)+(ctl_var/n)).reshape((m,1))
    t_stat=(int_mean-ctl_mean)/variance
    denom=np.power(variance,4)
    term1=(int_var/n)**2/(n-1)
    term1=term1.reshape((m,1))
    term2=(ctl_var/n)**2/(n-1)
    term2=term2.reshape((m,1))
    deg_of_free=denom/(term1+term2)
    return t_stat,deg_of_free
    
    

In [13]:
def calculate_p_value(interest,control):
    """
    Take in two dataframes of same shape (m x n), compute unadjusted two-sided p_value (m x 1) using Welch-t-test,
    assuming unequal variance.
    """
    t_stat,deg_of_free=calculate_t_stat(interest,control)
    p_value=stats.t.sf(np.abs(t_stat),deg_of_free)*2
    return p_value

In [14]:
def calculate_adjusted_p_value(interest,control):
    """
    Take in two dataframes of same shape (m x n), compute unadjusted two-sided p_value (m x 1) using Welch-t-test,
    and adjusted p_value (m x 1) using Benjamini-Hochberg method.
    """
    p_value=calculate_p_value(interest,control)
    m=len(p_value)
    p_tmp=pd.DataFrame(p_value,columns=['p-value'])
    p_tmp=p_tmp.sort_values('p-value')
    p_rank=stats.rankdata(p_tmp['p-value'])
    p_test=p_tmp['p-value']*m/p_rank
    p_adj=np.zeros(m)
    for i in range(m):
        p_adj[i]=np.min([np.min(p_test[i:]),1])   #double min, the outer min ensures p value doesn't exceed 1.
    p_tmp['p-adjusted']=p_adj
    p_tmp=p_tmp.sort_index()
    return np.round(p_tmp['p-value'],3),np.round(p_tmp['p-adjusted'],3)
        

In [25]:
results=df[['geneID','Locus Tag','Gene Symbol','geneProduct','WT_mean','185_mean','log2(FC)']]
results=results.reset_index(drop=True)
results['p-value'],results['p-value adj.']=calculate_adjusted_p_value(df[['185-1','185-2','185-3']],df[['WT-1','WT-2','WT-3']])

In [26]:
results.head(5)

,geneID,Locus Tag,Gene Symbol,geneProduct,WT_mean,185_mean,log2(FC),p-value,p-value adj.
0,650468844,Clo1313_0001,dnaA,chromosomal replication initiator protein DnaA,65.786667,13.213333,-2.315801,0.070,0.166
1,650468845,Clo1313_0002,NaN,"DNA polymerase III, beta subunit (EC 2.7.7.7)",64.206667,9.520000,-2.753690,0.054,0.145
2,650468846,Clo1313_0003,NaN,S4 domain protein YaaA,22.486667,8.810000,-1.351856,0.086,0.185
3,650468847,Clo1313_0004,recF,DNA replication and repair protein RecF,14.533333,0.780000,-4.219748,0.041,0.127
4,650468848,Clo1313_0005,NaN,hypothetical protein,243.493333,105.433333,-1.207551,0.001,0.030


### Check the fold change in key metabolic pathways

In [27]:
def check_pathway_transcription(file,df):
    pathway=pd.read_table(file)
    pathway=pathway[['Gene ID','Locus Tag','Gene Product Name']]
    out=pathway.merge(df,how='left',on='Locus Tag')
    out=out[['Gene ID','Locus Tag','Gene Product Name','Gene Symbol','WT_mean','185_mean','log2(FC)','p-value','p-value adj.']]
    return out

In [28]:
check_pathway_transcription('ketoacid_pathway(PWY-7111).txt',results)

,Gene ID,Locus Tag,Gene Product Name,Gene Symbol,WT_mean,185_mean,log2(FC),p-value,p-value adj.
0,650468944,Clo1313_0099,"acetolactate synthase, large subunit (EC 2.2.1...",NaN,274.333333,1965.026667,2.840547,0.002,0.041
1,650468945,Clo1313_0100,"acetolactate synthase, small subunit (EC 2.2.1...",NaN,295.116667,1796.410000,2.605759,0.000,0.019
2,650468946,Clo1313_0101,ketol-acid reductoisomerase (EC 1.1.1.86),ilvC,604.156667,7170.100000,3.568999,0.005,0.054
3,650469149,Clo1313_0304,dihydroxy-acid dehydratase,ilvD,301.216667,579.973333,0.945185,0.073,0.169
4,650469150,Clo1313_0305,"acetolactate synthase, large subunit (EC 2.2.1...",NaN,370.780000,2555.676667,2.785070,0.000,0.017
5,650470642,Clo1313_1798,acetaldehyde dehydrogenase (EC 1.2.1.10)/alcoh...,NaN,2242.053333,2422.586667,0.111728,0.402,0.520


In [ ]:
# check_pathway_transcription('mixedacid_pathway(FERMENTATION-PWY).txt',df)

In [ ]:
# check_pathway_transcription("reductiveTCAcycle(reductive TCA cycle I).txt",df)

In [ ]:
# check_pathway_transcription('glycolysis(glycolysis I).txt',df)

In [ ]:
# check_pathway_transcription('carbonhydrate_metabolism.txt',df)

In [ ]:
# check_pathway_transcription('valine_biosynthesis.txt',df)

In [ ]:
# df_filtered.sort_values('log2(FC)',ascending=False)

In [ ]:
# df[df['Locus Tag']=='Clo1313_0020']

### List of most differentially transcribed genes

In [30]:
results.sort_values('log2(FC)',ascending=False).head(10)

,geneID,Locus Tag,Gene Symbol,geneProduct,WT_mean,185_mean,log2(FC),p-value,p-value adj.
451,650469348,Clo1313_0501,NaN,Pectate lyase/Amb allergen,2.946667,89.773333,4.929131,0.010,0.070
1976,650471035,Clo1313_2199,NaN,iron-sulfur binding protein,0.573333,15.196667,4.728237,0.047,0.137
2138,650471237,Clo1313_2427,NaN,transport system permease protein,181.866667,3234.080000,4.152402,0.006,0.058
411,650469305,Clo1313_0458,rpsZ,ribosomal protein S14,26.940000,338.896667,3.653024,0.076,0.174
87,650468946,Clo1313_0101,ilvC,ketol-acid reductoisomerase (EC 1.1.1.86),604.156667,7170.100000,3.568999,0.005,0.054
713,650469653,Clo1313_0812,NaN,Domain of unknown function DUF3298,91.910000,806.266667,3.132963,0.008,0.064
663,650469595,Clo1313_0745,NaN,Uncharacterized conserved protein UCP033563,35.236667,255.480000,2.858061,0.002,0.042
85,650468944,Clo1313_0099,NaN,"acetolactate synthase, large subunit (EC 2.2.1.6)",274.333333,1965.026667,2.840547,0.002,0.041
1975,650471034,Clo1313_2198,hcp,hybrid cluster protein,2.576667,18.070000,2.810019,0.055,0.145
261,650469150,Clo1313_0305,NaN,"acetolactate synthase, large subunit (EC 2.2.1.6)",370.780000,2555.676667,2.785070,0.000,0.017


In [31]:
results.sort_values('log2(FC)').head(10)

,geneID,Locus Tag,Gene Symbol,geneProduct,WT_mean,185_mean,log2(FC),p-value,p-value adj.
1269,650470260,Clo1313_1416,NaN,response regulator receiver protein,207.856667,1.010000,-7.685090,0.003,0.046
1264,650470255,Clo1313_1411,NaN,CheA signal transduction histidine kinase,92.906667,0.636667,-7.189100,0.006,0.059
1679,650470706,Clo1313_1866,NaN,hypothetical protein,109.580000,0.846667,-7.015975,0.055,0.146
2014,650471079,Clo1313_2251,NaN,hypothetical protein,140.786667,1.170000,-6.910858,0.023,0.094
879,650469832,Clo1313_0991,NaN,"two component transcriptional regulator, LuxR ...",387.713333,3.606667,-6.748180,0.002,0.039
2620,650471767,Clo1313_2980,NaN,hemerythrin-like metal-binding protein,611.766667,5.883333,-6.700204,0.051,0.141
1266,650470257,Clo1313_1413,NaN,"MCP methyltransferase, CheR-type",107.100000,1.103333,-6.600946,0.003,0.049
1028,650469998,Clo1313_1160,ldh,L-lactate dehydrogenase (EC 1.1.1.27),300.943333,3.333333,-6.496382,0.024,0.096
2022,650471087,Clo1313_2259,NaN,methyl-accepting chemotaxis sensory transducer,23.413333,0.296667,-6.302344,0.024,0.097
909,650469864,Clo1313_1023,NaN,hypothetical protein,70.323333,0.936667,-6.230324,0.011,0.071


### Making a heat map (creating a customary color map)
After some thought, I think heat map is not a good idea to present this data, as there is only one condition. Without showing repeat samples, it will simply be a heat map of one column (pretty much defeats the purpose of a heat map.)
I leave the code here just for my future reference.

In [ ]:
test_plot=df.iloc[:10,[2,4,5,6,7,8,9]]

In [ ]:
test_plot

In [ ]:
gene_names=list(test_plot.iloc[:,0])

In [ ]:
# some exercise for making a heat map in ipython notebook. In this data set, as there is only one condition, 
# it is better to simply present data as fold change instead of showing the heatmap

cdict1 = {'red':   ((0.0, 0.0, 0.0),
                   (0.5, 0.0, 0.0),
                   (1.0, 1.0, 1.0)),

         'green': ((0.0, 1.0, 1.0),
                   (0.5, 0.0, 0.0),
                   (1.0, 0.0, 0.0)),

         'blue':  ((0.0, 0.0, 0.0),
                   (0.5, 0.0, 0.0),
                   (1.0, 0.0, 0.0))
        }
green_red=LinearSegmentedColormap('green_red',cdict1)
fig=plt.figure(figsize=(10,5))
plt.imshow(test_plot.iloc[:,1:],cmap=green_red)
plt.colorbar()
plt.grid(False)
ax=fig.gca()
ax.set_yticklabels(gene_names)
ax.tick_params(labelsize=20)
ax.set_yticks(range(10))
ax.set_xticks(range(6))